<a href="https://colab.research.google.com/github/Christinastalin/mrsapp/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

100%|██████████| 8.89M/8.89M [00:00<00:00, 57.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tmdb/tmdb-movie-metadata/versions/2


Types of recommender systems:
1. Content based
2. Collaborative Filtering
3. Hybrid

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
movies = pd.read_csv(path + "/tmdb_5000_movies.csv")
credits = pd.read_csv(path + "/tmdb_5000_credits.csv")

##Data Processing

In [72]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [73]:
#Merging both dataframes
movies = movies.merge(credits,on='title')

In [74]:
movies.shape

(4809, 23)

In [75]:
#Filtering relevant columns and dropping the rest
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [76]:
movies.shape

(4809, 7)

In [77]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [78]:
movies.isnull().sum()

,0
movie_id,0
title,0
overview,3
genres,0
keywords,0
cast,0
crew,0


In [79]:
movies.dropna(inplace= True)

In [12]:
movies.duplicated().sum()

np.int64(0)

In [80]:
#Converting the genres column in required format
import ast

def convert(object):
  L=[]
  for obj in ast.literal_eval(object):
      L.append(obj['name'])
  return L



In [81]:
movies['genres'] = movies['genres'].apply(convert)

In [15]:
movies['genres']

,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4804,"[Action, Crime, Thriller]"
4805,"[Comedy, Romance]"
4806,"[Comedy, Drama, Romance, TV Movie]"
4807,[]


In [82]:
movies['keywords']= movies['keywords'].apply(convert)

In [83]:
def convert3(obj):
  counter = 0
  L=[]
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [84]:
movies['cast']= movies['cast'].apply(convert3)

In [85]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
      if i['job']== 'Director':
        L.append(i['name'])
        break
  return L

In [86]:
movies['crew']= movies['crew'].apply(fetch_director)

In [87]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [88]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [89]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [90]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [91]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [24]:
#movies['genres']= movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [25]:
#movies['keywords']= movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
#movies['cast']= movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [27]:
#movies['crew']= movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [92]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [93]:
movies['tags']= movies['overview']+ movies['genres'] + movies['cast'] + movies['crew'] + movies['keywords']

In [94]:
new_df= movies[['movie_id', 'title' , 'tags']]

In [98]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

<ipython-input-98-e498c0e361a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [99]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [43]:
new_df['tags']= new_df['tags'].apply(lambda x:[i.lower() for i in x])

<ipython-input-43-16f7d181df22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']= new_df['tags'].apply(lambda x:[i.lower() for i in x])


In [100]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [101]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 5000, stop_words ='english')


In [102]:
vectors= cv.fit_transform(new_df['tags']).toarray()

In [103]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [49]:
vectors.shape

(4806, 5000)

In [104]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [61]:
!pip install nltk

In [62]:
import nltk

In [63]:
from nltk.stem.porter import PorterStemmer
Ps = PorterStemmer()

In [105]:
def stem(text):
  y=[]

  for i in text.split():
    y.append(Ps.stem(i))
  return ",".join(y)

In [106]:
new_df['tags'].apply(stem)

,tags
0,"in,the,22nd,century,,a,parapleg,marin,is,dispa..."
1,"captain,barbossa,,long,believ,to,be,dead,,ha,c..."
2,"a,cryptic,messag,from,bond’,past,send,him,on,a..."
3,"follow,the,death,of,district,attorney,harvey,d..."
4,"john,carter,is,a,war-weary,,former,militari,ca..."
...,...
4804,"el,mariachi,just,want,to,play,hi,guitar,and,ca..."
4805,"a,newlyw,couple',honeymoon,is,upend,by,the,arr..."
4806,"""signed,,sealed,,delivered"",introduc,a,dedic,q..."
4807,"when,ambiti,new,york,attorney,sam,is,sent,to,s..."


In [107]:
from sklearn.metrics.pairwise import cosine_similarity

In [109]:
similarity = cosine_similarity(vectors)

In [110]:
similarity

array([[1.        , 0.08740748, 0.05827165, ..., 0.02418254, 0.02564946,
        0.        ],
       [0.08740748, 1.        , 0.06451613, ..., 0.02677398, 0.        ,
        0.        ],
       [0.05827165, 0.06451613, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02418254, 0.02677398, 0.02677398, ..., 1.        , 0.07071068,
        0.04836508],
       [0.02564946, 0.        , 0.        , ..., 0.07071068, 1.        ,
        0.05129892],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05129892,
        1.        ]])

In [111]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

np.int64(744)

In [115]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [116]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [117]:
import pickle

In [119]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))